In [1]:
from likelihood import Likelihood
from likelihood import pack_params
from likelihood import unpack_params
from likelihood import get_cosmo_params
from likelihood import cosmo_params_to_obj

from angular_power import define_cosmo

import numpy as np

In [2]:
lhood = Likelihood()

Initialising likelihood
Initialisation Complete


In [3]:
cosmo_params = get_cosmo_params(define_cosmo())
blbg = np.array([2.])
bint = np.array([1.])

In [4]:
all_param = pack_params(cosmo_params, blbg, bint)
all_param

Array([ 0.8159,  0.2589,  0.0486,  0.6774,  0.9667, -1.    ,  2.    ,
        1.    ], dtype=float64)

In [5]:
unpack_params(all_param)

(Array([ 0.8159,  0.2589,  0.0486,  0.6774,  0.9667, -1.    ], dtype=float64),
 Array(2., dtype=float64),
 Array(1., dtype=float64))

In [6]:
lhood.logL(all_param)

Array(-2368.92019942, dtype=float64)

In [7]:
lhood.logLgauss(all_param)

Array(-2360.69758583, dtype=float64)